In [132]:
import os
import json
import random
import string
from typing import Annotated, TypedDict, List, Sequence, Optional
from langchain_groq import ChatGroq 
from langgraph.graph import StateGraph, START, END, MessagesState
from langgraph.types import Command
from langchain_core.messages import BaseMessage,HumanMessage,SystemMessage,AIMessage
from langgraph.graph.message import add_messages
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.checkpoint.memory import InMemorySaver
from langchain_core.tools import tool
from langgraph.prebuilt import ToolNode, InjectedState
from dotenv import load_dotenv

In [133]:
load_dotenv()
api_key = os.getenv("GROQ_API_KEY")

In [134]:
llm = ChatGroq(model="llama-3.1-8b-instant")

In [135]:
class SupervisorState(MessagesState):
    """
        State for multi-agent system
    """   
    next_agent: str
    user_input: str
    user_intent: str
    complaint: str
    missing_info:  List[dict]
    complaint_in_progress: bool = False

# Supervisor Node

def supervisor_node(state:SupervisorState)->SupervisorState:

    if state.get('complaint_in_progress'):
        return Command(goto="Complaint Agent")
 
    system_prompt = """
    You are a supervisor. Classify the user query into one of: inquiry, complaint, retention.

    Rules:
    - If the user mentions cancelling, switching, leaving, or expresses strong frustration about continuing the service, classify as retention.
    - If the user complains but does not express intent to leave, classify as complaint.
    - If the user is asking for info or guidance, classify as inquiry.

    Examples:
    - "My laptop stopped working" -> complaint
    - "Show me more info on my TCL TV" -> inquiry
    - "I want to cancel my Netflix subscription" -> retention
    - "My smartphone battery is not charging" -> complaint
    - "I want to know the specs of my AirPods" -> inquiry
    - "I want to cancel my gym membership" -> retention
    - "My last order was delayed and I am frustrated, considering switching" -> retention
    - "The app crashes often, this is annoying" -> complaint
    - "Where can I see my last invoice?" -> inquiry

    Return **only** the intent.
    """
    
    response = llm.invoke([
        {"role":"system", "content":system_prompt},
        {"role":"user", "content":state["user_input"]}
        ])
    
    intent=response.content.strip().lower()
    state['user_intent']=intent

    if intent=="inquiry":
        return Command(goto="Inquiry Agent")
    elif intent=="complaint":
        return Command(goto="Complaint Agent")
    elif intent=="retention":
        return Command(goto="Retention Agent")
    else:
        return Command(goto="Fallback")


In [136]:
# Worker nodes

def inquiry_node(state:SupervisorState)->SupervisorState:
    return {'messages':[f"Inquiry Handled: {state['user_input']}"]}

def fallback(state:SupervisorState)->SupervisorState:
    return {'messages':[f"Sorry, I couldn't understand your message"]}

In [137]:
@tool
def nlu(user_input:str, complaint:Optional[str]="")->dict:
    """It extracts the entities from the user input"""

    # complaint=state.complaint if state.complaint else ''
    system_prompt=f"""
    Extract the following from the user message:
    - product
    - issue_type
    - purchase_date
    For you information: {complaint} already been captured.
    Respond ONLY with a valid JSON object in this exact format:
    {{
        "product": string or null,
        "issue_type": string or null,
        "purchase_date": string or null
    }}
    """
    response = llm.invoke([
        {'role':'system','content':system_prompt},
        {'role':'user','content':user_input}
    ])
    complaint = json.loads(response.content) 
    return {'complaint':complaint}

@tool
def ask_missing_info(missing_value:str)->dict:
    """
    It will ask the missing information to user to get the 
    remaining details missing from the user input
    """
   
    question = f"Could you please provide {missing_value}."

    return {"missing_info":question}

@tool
def create_ticket(complaint: dict)->dict:
    """
    Tool to create a complaint ticket.
    Generates a ticket ID like IG408C90.
    """
    prefix = ''.join(random.choices(string.ascii_uppercase,k=2))
    number1 = random.randint(100,1000)
    mid = ''.join(random.choices(string.ascii_uppercase))
    number2 = random.randint(10,100)
    ticket_id = f"{prefix}{number1}{mid}{number2}"

    ticket={
        "ticket_id":ticket_id,
        "status":"created",
        "details":complaint
    }
    
    return ticket
    

In [ ]:
def complaint_node(state:SupervisorState)->SupervisorState:
   """
      This node handles complaint and helps to resolve complaint.  
   """
   extracted=nlu({'user_input':state['user_input']})
   print(extracted)
   missing_values=[k for k,v in extracted['complaint'].items() if v is None]
   print("Missing Values", missing_values)
   for missing_value in missing_values:
      missing_info = ask_missing_info({'missing_value':missing_value})
      
      print(missing_info)
   state['missing_info']=missing_info
   state['complaint_in_progress']=True

   print("creating ticket")
   ticket=create_ticket(extracted)
   print(ticket)

   return state

In [140]:
graph = StateGraph(SupervisorState)
graph.add_node("Supervisor",supervisor_node)
graph.add_node("Inquiry Agent",inquiry_node)
graph.add_node("Complaint Agent",complaint_node)
graph.add_node("Retention Agent",retention_node)
graph.add_node("Fallback", fallback)

graph.add_edge(START, "Supervisor")

app = graph.compile()

In [141]:
response = app.invoke({"user_input":"My mouse is not working."})

{'complaint': {'product': 'mouse', 'issue_type': 'not working', 'purchase_date': None}}
Missing Values ['purchase_date']
{'missing_info': 'Could you please provide purchase_date.'}
creating ticket
{'ticket_id': 'UY682C83', 'status': 'created', 'details': {'product': 'mouse', 'issue_type': 'not working', 'purchase_date': None}}


In [44]:
response = app.invoke({"user_input":"Today."})
response

{'messages': [HumanMessage(content='Inquiry Handled: Today.', additional_kwargs={}, response_metadata={}, id='c78f8213-7881-4c20-a510-39f35e6cd737')],
 'user_input': 'Today.'}

In [9]:
@tool
def churn_score(user_input:str)->int:
    """
        Calculate a churn risk score from user input.
        Returns category: high, medium, or low.
    """

    system_prompt = """ 
    You are a churn detection assistant.
    Based on the user input, classifiy their churn risk into:
    - high: user explicitly wants to cancel, switch, or sounds very frustrated.
    - medium: user shows dissatisfaction but hasn't decided to cancel yet.
    - low: user just asking questions or mild complaints.

    Examples:
    User: "I'm cancelling this useless service today."
    Churn risk: high

    User: "Your prices keep going up, I don't know if it's worth it anymore."
    Churn risk: medium

    User: "How do I cancel if I ever need to in the future?"
    Churn risk: low

    Respond with only one of: high, medium, low.
    """
    score = llm.invoke([
        {'role':'system', 'content':system_prompt},
        {'role':'user', 'content':user_input}
    ]).content

    return str(score)

@tool
def loyalty_score(score:str)->str:
    """ 
        Provide rewards to user based on churn score. 
    """
    reward_weights = {'high': 1.0, "medium": 0.6, "low": 0.2}
    clv_values = {"high": 1000, "medium": 500, "low": 200}
    clv_tier=random.choices(
        ['high','medium','low'],
        weights=[0.25,0.45,0.3],
        k=1
    )[0]

    reward_score = reward_weights[score]*clv_values[clv_tier]

    return reward_score

@tool
def loyalty_rewards(user_input:str, reward_score:float, churn_score:str)->str:
    """
    Generate a personalized retention message.
    """
    system_prompt=""" 
    You are a customer retention assistant. 
    Generate a polite, empathetic message based on the following:
    - The user's message
    - Their churn risk (high, medium, low)
    - Their loyalty score (40-1000)

    Rules:
    - If loyalty score >= 800 → emphasize strong appreciation and give a high reward (e.g., big discount, free premium month).
    - If 500-799 → show gratitude and offer a medium reward (e.g., discount or perk).
    - If 200-499 → acknowledge their value and give a small reward (e.g., loyalty points or small discount).
    - If < 200 → do not give a reward, just apologize and promise to improve.
    - If churn risk = high → always start by apologizing and showing empathy before mentioning any reward.
    - Keep the message short, friendly, concise and natural. Do not include technical terms or scores.
    
    Respond with only the final concise message.
    """

    user_context = f"""
    User message: {user_input}
    Churn risk: {churn_score}
    Loyalty score: {reward_score}
    """

    response = llm.invoke([{'role':'system', 'content':system_prompt},
                           {'role':'user','content':user_context}]).content
    
    return str(response)


In [10]:
def retention_node(state:SupervisorState)->SupervisorState:
    score = churn_score(state['user_input'])
    rewards = loyalty_score(score)
    loyalty=loyalty_rewards({'user_input':state['user_input'],'reward_score':rewards, 'churn_score':score})
    print(rewards)
    print(loyalty)
    return {'messages':[f"Retention Handled: {state['user_input']}"]}